In [2]:
import pandas as pd
import numpy as np
import json
from urllib import request

Class Roster

In [7]:
subjects_url = 'https://classes.cornell.edu/api/2.0/config/subjects.json?roster=SP22'
subjects_json = json.loads(request.urlopen(subjects_url).read())
subject_list = [x['value'] for x in subjects_json['data']['subjects']]

In [114]:
courses = pd.DataFrame(columns = ["Dept", "Number", "Course_Name", "Professor","Start_Time", "End_Time", "Location","Description","Pre/coreqs","Offered"])
def getFineData(row): return row['enrollGroups'][0]['classSections'][0]['meetings']
def getProf(row): 
  x = lambda r: r[0]['instructors'][0]['firstName'].strip() + ' '+r[0]['instructors'][0]['lastName'].strip()
  try: 
    return x(row)
  except: return ""
def getData(fineData, obj): 
  try:
    x = lambda r: r[0][obj]
    return x(fineData)
  except: return ""

for subject in subject_list: 
    url = f"https://classes.cornell.edu/api/2.0/search/classes.json?roster=SP22&subject={subject}"
    data_json = json.loads(request.urlopen(url).read())
    df = pd.DataFrame(data_json['data']['classes'])

    df['prof'] = df.apply(lambda row: getProf( getFineData(row)), axis=1)
    df['startTime'] = df.apply(lambda row: getData(getFineData(row), 'timeStart'), axis=1)  
    df['endTime'] = df.apply(lambda row: getData(getFineData(row), 'timeEnd'), axis=1)  
    df['location'] = df.apply(lambda row: getData(getFineData(row), 'facilityDescr'), axis=1)  

    df = df[["subject", "catalogNbr", "titleShort", "prof", "startTime", "endTime", "location", "description", "catalogPrereqCoreq", "catalogWhenOffered"]]
    df = df.rename(columns={"subject": "Dept", 'catalogNbr':"Number", "titleShort":"Course_Name", "prof":"Professor", "startTime":"Start_Time", "endTime":"End_Time", "location": "Location", "description":"Description", "catalogPrereqCoreq":"Pre/coreqs", "catalogWhenOffered":"Offered"})

    courses = courses.merge(df, how='outer') 

courses.to_csv('cleaned_medians.csv')

CUreviews

In [ ]:
data = df.to_numpy()

url = 'https://www.cureviews.org/v2/getCourseByInfo/'

difficulty = []
ratings = []
workloads = []

for val in data:

    try:
        sub = val[1].lower()
        num = int(val[2])
        myobj = {'number': num, 'subject': sub}

        res = requests.post(url, json = myobj).json()['result']

        diff = res['classDifficulty']
        rating = res['classRating']
        workload = res['classWorkload']
        
    except:
        diff = None
        rating = None
        workload = None
    
    difficulty += [diff]
    ratings += [rating]
    workloads += [workload]
        